In [19]:
# from google.colab import drive
# drive.mount("/gdrive", force_remount=True)
# # drive.mount('/gdrive')


Mounted at /gdrive


In [1]:
# %cd /gdrive
# %cd 'My Drive'
# # %cd Maga
# %cd Diploma

In [3]:
!ls

morewiki_1456.json  morewiki_50.json
morewiki_20.json    ranking_experiments.ipynb


In [0]:
# !git clone https://github.com/huggingface/transformers.git
# # !git clone https://github.com/huggingface/transformers --branch quickstart-model2model
# %cd transformers
# !pip3 install .
# !git pull
# !pip3 install --upgrade .

In [0]:
# ! pip3 uninstall transformers

In [2]:
# # !git clone https://github.com/huggingface/transformers
# %cd transformers
# !pip3 install .

In [107]:
import json
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [0]:
# !tar -xzf rubert_cased_L-12_H-768_A-12_pt.tar.gz

In [6]:
# import os
# os.chdir("../")

In [7]:
!ls
# !cd rubert_cased_L-12_H-768_A-12_pt

morewiki_1456.json  morewiki_50.json
morewiki_20.json    ranking_experiments.ipynb


In [109]:
# ru_bert_path = "rubert_cased_L-12_H-768_A-12_pt"
ru_bert_path = "/datadrive/axon/something_reasoning/models/rubert_cased_L-12_H-768_A-12_pt/"

In [110]:
# relations = ["синонимы", "индикатор", "заголовок статьи", "краткое описание", "статья", "documents"]
relations = ["синонимы", "labels", "sitelinks", "summary", "content"]
# "sitelinks" gives 1

In [111]:
tokenizer = BertTokenizer.from_pretrained(ru_bert_path)

In [112]:
model = BertModel.from_pretrained(ru_bert_path)

In [113]:
model = model.eval()

In [114]:
from tqdm import tqdm

In [115]:
def default_collate(batch):
    return {k: torch.LongTensor(v) for k, v in batch.items()}

In [117]:
def get_vec(text):

#     print("get_vec, text", text)

    b = default_collate(tokenizer.batch_encode_plus([text]))
    out = model(**b)
    
    vec = out[1][0]
    
    return vec.detach().numpy()

In [175]:
def get_cosins(query_text, info):
    query = get_vec(query_text)
    example = []
    
    # count = 0

    for q in info["ranking_candidates"]:

        vectors = []
        
        # for answer
        
        try:
            if q["ans_cont"] != "": #changed, rubert was just text = q["ans_cont"]
                text = q["ans_cont"]
            else:
                text = q["ans_sum"]
        except:
            try:
                text = q["ans_sum"]
            except:
                text = ''
        val_vec_ans = get_vec(text)
        vectors.append(val_vec_ans)
        
        # for graph info
        for val in q["graph"]:

            if val["relation"] in relations:
                text = val["value"] if isinstance(val["value"], str) else " ".join(val["value"])
                text = text[:256]
                val_vec = get_vec(text)
                vectors.append(val_vec)
                
        # for doc in q["documents"]:
        #     text = doc["document"][:256]
        #     val_vec = get_vec(text)
        #     vectors.append(val_vec)
        
        v = np.vstack(vectors)
        qv = np.vstack([query] * len(v))

        example.append({"id": q["Q"], "cosins": cosine_similarity(v, qv)[:, 0]})
        
#         #for one + graph answer
#     vectors1 = []
#     text1 = info["ans_g"]
# #     text1 = "лалалалалалалала"
#     val_vec1 = get_vec(text1)
#     vectors1.append(val_vec1)
#     v1 = np.vstack(vectors1)
#     qv1 = np.vstack([query] * len(v))
#     example.append({"id": "ans_g", "cosins": cosine_similarity(v1, qv1)[:, 0]})
    

    return example

In [173]:
# wikilink_final/morewiki_20.json
with open("all_wiki_1810r21rubert.json", "r") as file:
    data = json.load(file)

In [127]:
from tqdm import tqdm_notebook

In [128]:
def get_cosins_all(data):
    lines = []
    y_trues = []
    questions = [] #for Qs
    for query_text in tqdm_notebook(data, total=len(data), leave=False):
        
        questions.append(query_text) # for Qs
#         print("get_cosins_all, query_text",query_text)
        info = data[query_text]
#         print("info",info)
        y_trues.append(info.get("y_gold"))
#         print("get_cosins_all, y_trues",y_trues)
        lines.append(get_cosins(query_text, info))
    return lines, y_trues, questions

In [121]:
def get_scores(lines, y_trues, top_n=2):
    answers = 0
    all_counter = 0
    zero_counts = 0
    lens_ranking = []

    for line, y_true in zip(lines, y_trues):
        cosins = []
        ids = []
        for q in line:
            cosins.append(max(q["cosins"]))
            ids.append(q["id"])
        lens_ranking.append(len(cosins))
        if cosins:
            idx = np.argsort(cosins)[-top_n:] # ?
            if y_true is not None:
                all_counter += 1
                if y_true in np.array(ids)[idx]:
                    answers += 1
        else:
            zero_counts += 1
    return answers, all_counter, zero_counts, lens_ranking

In [129]:
def get_top_5(lines, y_trues, questions, top_n=2):
    answers = 0
    all_counter = 0
    zero_counts = 0
    lens_ranking = []
    question_candidates = {}

    for line, y_true, question in zip(lines, y_trues, questions):
        cosins = []
        ids = []
        for q in line:
            cosins.append(max(q["cosins"]))
            ids.append(q["id"])
        lens_ranking.append(len(cosins))
        if cosins:
            idx = np.argsort(cosins)[-top_n:]
            if y_true is not None:
                all_counter += 1
                top_Q = np.array(ids)[idx]
                question_candidates[question] = top_Q
                if y_true in np.array(ids)[idx]:
                    answers += 1
        else:
            zero_counts += 1
    return answers, all_counter, zero_counts, lens_ranking, question_candidates

In [45]:
# with open("ranking_data_v1-1.json", "r") as file:
#     data = json.load(file)

In [46]:
# with open("res4_wikidata.json", "r") as file:
#     data = json.load(file)

In [47]:
!ls

link_ranked1456.json  ranking_all_questions-eval.ipynb
morewiki_1456.json    ranking_answers.ipynb
morewiki_20.json      ranking_experiments.ipynb
morewiki_50.json


In [176]:
lines, y_trues, questions = get_cosins_all(data)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [49]:
# y_trues

In [71]:
# lines

In [177]:
answers, all_counter, zero_counts, lens_ranking, question_candidates = get_top_5(lines, y_trues, questions, top_n=21)


In [170]:
# question_candidates

In [178]:
type(question_candidates)

dict

In [179]:
for q in question_candidates:
    if type(question_candidates[q]) == list:
        continue
    question_candidates[q] = question_candidates[q].tolist()
#     print(question_candidates[q])
#     break

In [165]:
np.array([[1,2,3],[4,5,6]]).tolist()

[[1, 2, 3], [4, 5, 6]]

In [190]:
# import json

# with open("Q_for_R_rubert_answ_ranked_ginfo---.json", "w", encoding="utf-8") as f:
#         json.dump(question_candidates, f, ensure_ascii=False, indent=4)

In [132]:
# import json

# with open("link_ranked1456--.json", "w", encoding="utf-8") as f:
#         json.dump(question_candidates, f, ensure_ascii=False, indent=4)

In [69]:
# lens_ranking

### For 1810 questions, 2-21 Qs per question to rank - RuBERT + graph info

In [186]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=1)


In [187]:
answers / len(lines) 

0.4867403314917127

In [184]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=2)


In [185]:
answers / len(lines) 

0.6591160220994475

In [182]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=3)


In [183]:
answers / len(lines) 

0.7591160220994475

In [188]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=5)


In [189]:
answers / len(lines)

0.888950276243094

### For 1810 questions, 2-21 Qs per question to rank - MULTI-BERT

In [154]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=1)


In [155]:
answers / len(lines) 

0.46353591160220997

In [156]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=2)


In [157]:
answers / len(lines) 

0.6353591160220995

In [158]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=3)


In [159]:
answers / len(lines) 

0.7309392265193371

In [160]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=5)


In [161]:
answers / len(lines) 

0.850828729281768

### For 1810 questions, 2-21 Qs per question to rank - RUBERT

In [137]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=1)


In [138]:
answers / len(lines) # labels 0.28 sitelinks 0.22 all 0.28 for 50 / rubert 1810 0.45 with 345 true (one) Q

0.4580110497237569

In [139]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=2)

In [140]:
answers / len(lines) #labels 0.48 sitelinks 0.54 all 0.5 for 50 / rubert 1810 0.64 with 345 true (one) Q

0.6353591160220995

In [141]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=3)

In [142]:
answers / len(lines) #labels 0.66 sitelinks 0.64 all 0.7 for 50 / rubert 1810 0.74 with 345 true (one) Q

0.7397790055248619

In [149]:
answers, all_counter, zero_counts, lens_ranking = get_scores(lines, y_trues, top_n=5)

In [150]:
answers / len(lines)

0.8569060773480663

In [63]:
answers, all_counter, zero_counts, lens_ranking, question_candidates = get_top_5(lines, y_trues, questions, top_n=2)

In [101]:
question_candidates

{'Кем был Ариф Васильевич Сапаров?': array(['Q4312193', 'Q4408035'], dtype='<U8'),
 'Кем был Константин Николаевич Асеев?': array(['Q4071207', 'Q4262989'], dtype='<U8'),
 'В каком году произошла отмена крепостного права в России?': array(['Q2300774', 'Q4127088'], dtype='<U9'),
 'Кем по профессии был Александр Николаевич Мальский?': array(['Q28914', 'Q4279466'], dtype='<U8'),
 'Ефим Михайлович Чепцов — год рождения?': array(['Q4511929', 'Q1345004'], dtype='<U8'),
 'Кем по профессии был Александр Моисеевич Хамадан?': array(['Q519076', 'Q18405199'], dtype='<U9'),
 'В какой войне принимал участие Игорь Михайлович Бойцов?': array(['Q198', 'Q4090488'], dtype='<U8'),
 'В каком году родился Фёдор Гаврилович Гаврилов?': array(['Q15065435', 'Q4131200'], dtype='<U9'),
 'Беатрикс — дата рождения?': array(['Q813291', 'Q29574'], dtype='<U8'),
 'Каким годом датируется написание картины Мунка "Ревность"?': array(['Q625429', 'Q16939042'], dtype='<U9'),
 'В какой войне участвовал Теодор Рузвельт?': arra

In [53]:
answers / len(lines)

1.0

In [54]:
np.mean(lens_ranking)

5.96